<a href="https://colab.research.google.com/github/webbigdata-jp/python_sample/blob/main/ibm_granite_granite_3_0_8b_instruct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%sh
pip install torch torchvision torchaudio
pip install accelerate==0.34.0
pip install --upgrade "transformers>=4.45"
pip install -U bitsandbytes

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

device = "auto"
model_path = "ibm-granite/granite-3.0-8b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_path)
# drop device_map if running on CPU
model = AutoModelForCausalLM.from_pretrained(model_path, load_in_4bit=True, device_map=device)
model.eval()



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

GraniteForCausalLM(
  (model): GraniteModel(
    (embed_tokens): Embedding(49155, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-39): 40 x GraniteDecoderLayer(
        (self_attn): GraniteSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): GraniteMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=12800, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=12800, bias=False)
          (down_proj): Linear4bit(in_features=12800, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): GraniteRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): GraniteRMSNorm((4096,), eps=1e-05)
      )
   

In [3]:
# change input text as desired
chat = [
    { "role": "user", "content": "へいへいへーい！俺を驚かすようなビックリする事とそれを使った一流のジョークを頼むぜ、ビッグブラザー！" },
]
chat = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
# tokenize the text
input_tokens = tokenizer(chat, return_tensors="pt").to("cuda")
# generate output tokens
output = model.generate(**input_tokens,
                        max_new_tokens=300)
# decode output tokens into text
output = tokenizer.batch_decode(output)
model_marker = "model\n"
model_response = output[0].split(model_marker)[-1].strip()
print(model_response)

/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:452: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


<|start_of_role|>user<|end_of_role|>へいへいへーい！俺を驚かすようなビックリする事とそれを使った一流のジョークを頼むぜ、ビッグブラザー！<|end_of_text|>
<|start_of_role|>assistant<|end_of_role|>こんにちは！ビックリする事を一流のジョークに変えるには、創造性とタイミングが大切です。以下は、ビックリする事をジョークに変える方法の例です：

1. 期待を裏切る - 通常、ビックリする事は、期待を裏切ることで生まれます。ジョークでは、期待を裏切ってビックリする事を利用することができます。例えば、友達に「今日は何を食べるの？」と聞かれたら、「ビックリする事」と答えることができます。これは、期待を裏切ってビックリする事をジョークに変えることができます。
2. 反省する - ビックリする事は、反省の機会にもなります。ジョークでは、ビックリする事を反省して、何が原因でビックリする事になったのかを考えてみることができます。例えば、ビックリする事は、過度な努力や過度なストレスなどで起こることがあります。これをジョークに変えるには、過度な努力やストレスを減ら
